# CUMUL

This notebook is an attempt to recreate teh CUMUL solution for WFA proposed by XXXX in 2017?

The code was not provided, so this is an approximation based on the description in the paper.

# Pack order based encoder

In [ ]:
import numpy as np
from scapy.all import *

def is_inbound(packet):
    # Implement the function to check if the packet is inbound or not
    pass

def analyze_pcap(pcap_path):
    packets = rdpcap(pcap_path)
    num_samples = 100
    total_packets = len(packets)
    interval = total_packets / num_samples

    running_total = 0
    sampled_totals = []

    for i, packet in enumerate(packets):
        if is_inbound(packet):
            running_total += len(packet)
        else:
            running_total -= len(packet)

        if i % interval == 0:
            sampled_totals.append(running_total)

    # Convert the list of sampled totals to a NumPy array
    sampled_totals_array = np.array(sampled_totals)

    return sampled_totals_array

# Usage example
pcap_path = "path/to/your/pcap/file.pcap"
result_array = analyze_pcap(pcap_path)
print(result_array)


# Time based encoder

In [ ]:
import numpy as np
from scapy.all import *

def is_inbound(packet):
    # Implement the function to check if the packet is inbound or not
    pass

def analyze_pcap(pcap_path):
    packets = rdpcap(pcap_path)
    num_samples = 100
    pcap_duration = 20  # 20 seconds
    interval = pcap_duration / num_samples

    running_total = 0
    sampled_totals = []
    current_sample_time = 0

    for packet in packets:
        if is_inbound(packet):
            running_total += len(packet)
        else:
            running_total -= len(packet)

        packet_time = packet.time
        if packet_time >= current_sample_time:
            sampled_totals.append(running_total)
            current_sample_time += interval

            if len(sampled_totals) == num_samples:
                break

    # Convert the list of sampled totals to a NumPy array
    sampled_totals_array = np.array(sampled_totals)

    return sampled_totals_array

# Usage example
pcap_path = "path/to/your/pcap/file.pcap"
result_array = analyze_pcap(pcap_path)
print(result_array)


# Visualiser  

In [ ]:
import matplotlib.pyplot as plt

def plot_encoded_array(encoded_array):
    # Create the x axis values (sample numbers)
    x = range(len(encoded_array))

    # Create a line plot with the x and y values
    plt.plot(x, encoded_array)

    # Set the x and y axis labels
    plt.xlabel('Sample Number')
    plt.ylabel('Cumulative Size')

    # Add a title for the plot
    plt.title('Cumulative Size at Sample Points')

    # Display the plot
    plt.show()

# Usage example
# Assuming the encoded_array variable has already been generated
plot_encoded_array(encoded_array)


# ML Models

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

# Assuming the features and labels arrays have already been generated
features = np.array(...)  # The concatenated large array of shape (x, 100)
labels = np.array(...)    # The corresponding array of shape (x,)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train and evaluate the K-NN model
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

print("K-NN Model:")
print(confusion_matrix(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))

# Train and evaluate the SVM model
svm = SVC(kernel='linear', C=1)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

print("SVM Model:")
print(confusion_matrix(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))
